# 泛型

如果我们声明了一个函数类型，那它就会很难扩展：

```ts
function identity(x: number): number {
    return x
}
```

如果之后我们需要字符串类型的参数，那它就不能满足了。

- 一个方式是利用上一章的函数类型重载。
- 或者使用 any 类型

```ts
function identity(x: any): any {
    return x
}
```

但是使用 any 会丢失类型信息，一个更简便的方法是使用**泛型**。

In [1]:
function identity<T>(x: T): T {
    return x
}

undefined

泛型是一个类型参数，此处我们添加了 `T` 类型参数，说明我们的函数参数都是 T 类型的，返回值和参数值类型相同。

#### 我们可以显示地传入泛型类型参数

In [2]:
identity<number>(1)

1

#### 也可以利用类型推到，让编译器查看参数类型自行决定泛型类型。

In [3]:
identity('hello')

'hello'

## 泛型变量

使用泛型变量时，这些参数被视为任意或者所有类型，下边的格式被认为是错误的，因为某些类型可能不支持 `+` 操作符：

*但是 any 类型可以使用 `+`?*

In [4]:
function add<T>(x: T, y: T): T {
    return x + y
}

Error: Line 2, Character 12
    return x + y
___________^
TS2365: Operator '+' cannot be applied to types 'T' and 'T'.

## 泛型类型

#### 声明一个泛型函数类型

In [5]:
let myFunc: <T>(x: T) => T = function<U>(m: U): U {
    return m
}

myFunc(9)

9

#### 利用接口进行声明

In [6]:
interface GenericsFunc {
    <T>(x: T): T
}

let same: GenericsFunc = function <T>(x: T): T {
    return x
}

same(9)

9

我们也可以将泛型参数当做整个接口的一个参数：

In [7]:
interface StillGenericsFunc<T> {
    (x: T): T
}

let stillSame: StillGenericsFunc<number> = function <T>(x: T): T {
    return x
}

stillSame(9)

9

## 泛型类

泛型类和上边的泛型接口类似，在类名称后边接收一个泛型类型参数。

**类的泛型参数只对其实例有效，static 属性不能利用这一特性。**

In [8]:
class GenericNumber<T> {
    zeroValue: T
    add: (x: T, y: T) => T
}

let myGenericNumber = new GenericNumber<number>()
myGenericNumber.zeroValue = 0
myGenericNumber.add = function(x, y) {
    return x+y
}


let stringGenericNumber = new GenericNumber<string>()
stringGenericNumber.zeroValue = ''
stringGenericNumber.add = function(x, y) {
    return x+y
}

console.log(stringGenericNumber.add('1', '2'))

12


undefined

## 泛型约束

上边的[泛型变量](#泛型变量)例子提到，对两个泛型类型的对象进行`+`操作符会报错，因为泛型类型没有支持该操作符。相比于任意类型，我们可以约束我们的泛型：只要传入的类型具有某个属性，我们就允许。比如下例，因为泛型缺少迭代协议：

In [9]:
function loopAndReturn<T>(x: T): T {
    for (let y of x) {}
    return x
}

Error: Line 2, Character 19
    for (let y of x) {}
__________________^
TS2488: Type 'T' must have a '[Symbol.iterator]()' method that returns an iterator.

#### 我们约束我们的泛型只接受实现了迭代协议的类型

In [10]:
interface Loopable {
    [Symbol.iterator]: () => { next: () => { value: any; done: boolean } }
}
function loopAndReturn<T extends Loopable>(x: T): T {
    for (let y of x) {
        console.log(y)
    }
    return x
}

loopAndReturn([1, 2, 3])

1
2
3


[ 1, 2, 3 ]

In [11]:
loopAndReturn(9)

Error: Line 1, Character 15
loopAndReturn(9)
______________^
TS2345: Argument of type '9' is not assignable to parameter of type 'Loopable'.

#### 在泛型约束中使用类型参数

使用一个泛型来约束另一个泛型：

In [12]:
function getValue<T, K extends keyof T>(obj: T, key: K) {
    return obj[key]
}

let obj = {
    a: 1,
    b: 2,
}
getValue(obj, 'a')

1

In [13]:
getValue(obj, 'x')

Error: Line 1, Character 15
getValue(obj, 'x')
______________^
TS2345: Argument of type '"x"' is not assignable to parameter of type '"a" | "b"'.